In [1]:
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
!pip install transformers accelerate bitsandbytes -q


from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

In [3]:
model_name_or_id = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"


model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

In [4]:
def text_gen(prompt):
  prompt = f"### Human:{prompt}\n### Assistant:"


  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

  generation_config = GenerationConfig(
      do_sample=True,
      top_k=1,
      temperature=0.99,
      max_new_tokens=600,
      pad_token_id=tokenizer.eos_token_id
  )


  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [5]:
text_gen("سه قاتل در یک اتاق هستند.. یکی وارد اتاق می شود و یکی از آنها را می کشد. هیچ کس اتاق را ترک نمی کند چند قاتل در اتاق باقی مانده است؟ استدلال خود را توضیح دهید")

### Human:سه قاتل در یک اتاق هستند.. یکی وارد اتاق می شود و یکی از آنها را می کشد. هیچ کس اتاق را ترک نمی کند چند قاتل در اتاق باقی مانده است؟ استدلال خود را توضیح دهید
### Assistant: این یک سوال کلاسیک است! پاسخ این است که تعداد قاتل ها در اتاق باقی مانده، همان تعداد قاتل ها است که در ابتدا در اتاق بودند. دلیل این است که یکی از قاتل ها وارد اتاق می شود و یکی از آنها را می کشد، اما هیچ کس اتاق را ترک نمی کند. بنابراین، تعداد قاتل ها در اتاق باقی مانده، همان تعداد قاتل ها است که در ابتدا در اتاق بودند.


In [6]:
text_gen('ملکه الیزابت کیست؟')

### Human:ملکه الیزابت کیست؟
### Assistant:ملکه الیزابت یک شاهزاده بریتانیایی است که از ۱۹۵۲ تا ۲۰۲۲ بر تخت سلطنت بریتانیا سلطنت کرد. او در ۲۱ آوریل ۱۹۲۶ در ولز به دنیا آمد و در ۸ سپتامبر ۲۰۲۲ در سن ۹۶ سالگی درگذشت. ملکه الیزابت دوم طولانی‌ترین سلطنت بریتانیا را در تاریخ خود داشت و در طول سلطنت خود بریتانیا را از جنگ جهانی دوم تا جنگ سرد و سپس به دوران مدرن و جهانی شدن رهبری کرد. او همچنین در طول سلطنت خود به عنوان یک نماد ملی و یک نماد برای بریتانیا شناخته شد. ملکه الیزابت دوم همچنین در طول سلطنت خود به عنوان یک نماد ملی و یک نماد برای بریتانیا شناخته شد. او همچنین در طول سلطنت خود به عنوان یک نماد ملی و یک نماد برای بریتانیا شناخته شد. ملکه الیزابت دوم همچنین در طول سلطنت خود به عنوان یک نماد ملی و یک نماد برای بریتانیا شناخته شد. ملکه الیزابت دوم همچنین در طول سلطنت خود به عنوان یک نماد ملی و یک نماد برای بریتانیا شناخته شد. ملکه الیزابت دوم همچنین در طول سلطنت خود به عنوان یک نماد ملی و یک نماد برای بریتانیا شناخته شد. ملکه الیزابت دوم همچنین در طول سلطنت خود به عنوان یک نماد ملی و

In [7]:
text_gen('اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟')

### Human:اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Assistant:سلام! توپ در لیوان برعکس بود، بنابراین وقتی لیوان را برداشتیم و داخل مایکروویو گذاشتیم، توپ هنوز در لیوان برعکس است. مایکروویو فقط لیوان را گرم می کند، نه توپ را. بنابراین توپ هنوز در لیوان برعکس است.


In [19]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gazelle98/wikipedia-medical-question-answering")

print("Path to dataset files:", path)

100%|██████████| 625k/625k [00:00<00:00, 86.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/gazelle98/wikipedia-medical-question-answering/versions/1


In [10]:
import pandas as pd
import numpy as np
import os

In [20]:
os.listdir('/root/.cache/kagglehub/datasets/gazelle98/wikipedia-medical-question-answering/versions/1')

['1000.csv']

In [21]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/gazelle98/wikipedia-medical-question-answering/versions/1/1000.csv')
df.head()

,Unnamed: 0,question,answer,link,paragraph_text,text,title,categories
0,0,دلیل ایجاد اسهال چرب چیست؟,علت آن عدم جذب صحیح چربی‌ها در روده باریک است....,https://fa.wikipedia.org/wiki/%D8%A7%D8%B3%D9%...,اسهال به طور کلی، به عنوان افزایش وزن مدفوع (ب...,اسهال چرب (انگلیسی: Steatorrhea‎) یا چرب‌پیخال...,اسهال چرب,"['رده:Infobox medical condition', 'رده:اختلالا..."
1,1,عوارض اسهال چرب چیست؟,اسهال چرب، سبب ایجاد اختلال شدید در فرآیند جذب...,https://fa.wikipedia.org/wiki/%D8%A7%D8%B3%D9%...,اسهال چرب با حجم زیاد و براق از عوارض اسهال‌ها...,اسهال چرب (انگلیسی: Steatorrhea‎) یا چرب‌پیخال...,اسهال چرب,"['رده:Infobox medical condition', 'رده:اختلالا..."
2,2,درمان واریس مری چگونه انجام می‌شود؟,درمان با اتساع بالون یا تزریق وازوپرسین وریدی ...,https://fa.wikipedia.org/wiki/%D9%88%D8%A7%D8%...,درمان با اتساع بالون یا تزریق وازوپرسین وریدی ...,واریس مری (به انگلیسی: Esophageal varices)، وا...,واریس مری,"['رده:Infobox medical condition', 'رده:بیماری\..."
3,3,خطر اصلی و عوارض واریس مری چیست؟,خطر اصلی واریس مری، پارگی عروق متسع و خونریزی ...,https://fa.wikipedia.org/wiki/%D9%88%D8%A7%D8%...,واریس مری (به انگلیسی: Esophageal varices)، وا...,واریس مری (به انگلیسی: Esophageal varices)، وا...,واریس مری,"['رده:Infobox medical condition', 'رده:بیماری\..."
4,4,عوارض جانبی میزوپرستول چیست؟,سردرد، تهوع، کرامپ رحمی، دل درد، اسهال، نفخ و ...,https://fa.wikipedia.org/wiki/%D9%85%DB%8C%D8%...,سردرد، تهوع، کرامپ رحمی، دل درد، اسهال، نفخ و ...,میزوپروستول (به انگلیسی: Misoprostol) ترکیبی س...,میزوپرستول,"['رده:بیماری\u200cهای گوارشی', 'رده:داروهای ضر..."


In [22]:
df.shape

(1000, 8)

In [23]:
df.isnull().sum()

,0
Unnamed: 0,0
question,0
answer,0
link,0
paragraph_text,0
text,0
title,0
categories,0


In [24]:
df_b = df[:50]

In [25]:
df_b.head()

,Unnamed: 0,question,answer,link,paragraph_text,text,title,categories
0,0,دلیل ایجاد اسهال چرب چیست؟,علت آن عدم جذب صحیح چربی‌ها در روده باریک است....,https://fa.wikipedia.org/wiki/%D8%A7%D8%B3%D9%...,اسهال به طور کلی، به عنوان افزایش وزن مدفوع (ب...,اسهال چرب (انگلیسی: Steatorrhea‎) یا چرب‌پیخال...,اسهال چرب,"['رده:Infobox medical condition', 'رده:اختلالا..."
1,1,عوارض اسهال چرب چیست؟,اسهال چرب، سبب ایجاد اختلال شدید در فرآیند جذب...,https://fa.wikipedia.org/wiki/%D8%A7%D8%B3%D9%...,اسهال چرب با حجم زیاد و براق از عوارض اسهال‌ها...,اسهال چرب (انگلیسی: Steatorrhea‎) یا چرب‌پیخال...,اسهال چرب,"['رده:Infobox medical condition', 'رده:اختلالا..."
2,2,درمان واریس مری چگونه انجام می‌شود؟,درمان با اتساع بالون یا تزریق وازوپرسین وریدی ...,https://fa.wikipedia.org/wiki/%D9%88%D8%A7%D8%...,درمان با اتساع بالون یا تزریق وازوپرسین وریدی ...,واریس مری (به انگلیسی: Esophageal varices)، وا...,واریس مری,"['رده:Infobox medical condition', 'رده:بیماری\..."
3,3,خطر اصلی و عوارض واریس مری چیست؟,خطر اصلی واریس مری، پارگی عروق متسع و خونریزی ...,https://fa.wikipedia.org/wiki/%D9%88%D8%A7%D8%...,واریس مری (به انگلیسی: Esophageal varices)، وا...,واریس مری (به انگلیسی: Esophageal varices)، وا...,واریس مری,"['رده:Infobox medical condition', 'رده:بیماری\..."
4,4,عوارض جانبی میزوپرستول چیست؟,سردرد، تهوع، کرامپ رحمی، دل درد، اسهال، نفخ و ...,https://fa.wikipedia.org/wiki/%D9%85%DB%8C%D8%...,سردرد، تهوع، کرامپ رحمی، دل درد، اسهال، نفخ و ...,میزوپروستول (به انگلیسی: Misoprostol) ترکیبی س...,میزوپرستول,"['رده:بیماری\u200cهای گوارشی', 'رده:داروهای ضر..."


In [26]:
df_b['answer'][0]

'علت آن عدم جذب صحیح چربی\u200cها در روده باریک است. عدم جذب صحیح چربی\u200cها منجر به افزایش میزان چربی در مدفوع می\u200cشود. افزایش میزان چربی مدفوع، باعث ایجاد حالتی شل در مدفوع می\u200cشود.'

In [27]:
import re

In [38]:
def text_cleaning(text):
  text = re.sub('[\u200c]', ' ', text)
  text = re.sub('[\n]', ' ', text)
  text = re.sub('[a-zA-Z]', ' ', text)
  text = re.sub('\s+', ' ', text)
  return text.strip()

In [39]:
df_b['answer'] = df_b['answer'].apply(text_cleaning)
df_b['question'] = df_b['question'].apply(text_cleaning)

/tmp/ipython-input-39-698796758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b['answer'] = df_b['answer'].apply(text_cleaning)
/tmp/ipython-input-39-698796758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b['question'] = df_b['question'].apply(text_cleaning)


In [40]:
df_b['answer'][0]

'علت آن عدم جذب صحیح چربی ها در روده باریک است. عدم جذب صحیح چربی ها منجر به افزایش میزان چربی در مدفوع می شود. افزایش میزان چربی مدفوع، باعث ایجاد حالتی شل در مدفوع می شود.'

In [41]:
### Human:{prompt}\n### Assistant:
b = []
for i in range(df_b.shape[0]):
  question_text = df_b["question"][i]
  answer_text = df_b["answer"][i]
  text = f"### Human:{question_text}\n### Assistant: {answer_text}"
  b.append(text)

In [42]:
b

['### Human:دلیل ایجاد اسهال چرب چیست؟\n### Assistant: علت آن عدم جذب صحیح چربی ها در روده باریک است. عدم جذب صحیح چربی ها منجر به افزایش میزان چربی در مدفوع می شود. افزایش میزان چربی مدفوع، باعث ایجاد حالتی شل در مدفوع می شود.',
 '### Human:عوارض اسهال چرب چیست؟\n### Assistant: اسهال چرب، سبب ایجاد اختلال شدید در فرآیند جذب ویتامین های محلول در چربی می شود. در نتیجه، چنین بیمارانی دچار کاهش میزان جذب ویتامین های محلول در چربی می شوند. این بیماران همچنین به علت از دست دادن خون و نیز عدم جذب مواد غذایی، مستعد ابتلاء به کم خونی هم هستند. همچنین این افراد به علت دردهای شکمی پس از خوردن غذا، از خوردن ترس و نفرت دارند.',
 '### Human:درمان واریس مری چگونه انجام می شود؟\n### Assistant: درمان با اتساع بالون یا تزریق وازوپرسین وریدی یا مواد اسکلروزان انجام می شود. در موارد عدم پاسخ به درمان از شنت پورتوکاوای خارج کبدی جهت کاهش فشار عروق خونریزی کننده استفاده می شود.',
 '### Human:خطر اصلی و عوارض واریس مری چیست؟\n### Assistant: خطر اصلی واریس مری، پارگی عروق متسع و خونریزی گوارشی است',
 '### Hu

In [43]:
text_gen('دلیل ایجاد اسهال چرب چیست؟')

### Human:دلیل ایجاد اسهال چرب چیست؟
### Assistant:دلیل ایجاد اسهال چرب چیست؟

اسهال چرب یا اسهال چرب و چرب، یک وضعیت پزشکی است که در آن مدفوع چرب و چرب وجود دارد. این وضعیت معمولاً به دلیل مشکلات گوارشی یا بیماری‌های خاص ایجاد می‌شود. برخی از دلایل ایجاد اسهال چرب عبارتند از:

1. بیماری‌های گوارشی: بیماری‌های گوارشی مانند التهاب روده، التهاب روده بزرگ، التهاب روده کوچک، و بیماری‌های روده بزرگ مانند کولیت اولسریو و کولیت اولسریو می‌توانند منجر به ایجاد اسهال چرب شوند.
2. بیماری‌های روده: بیماری‌های روده مانند کولیت اولسریو، کولیت اولسریو، و بیماری‌های روده بزرگ مانند کولیت اولسریو می‌توانند منجر به ایجاد اسهال چرب شوند.
3. عفونت‌های روده: عفونت‌های روده مانند Salmonella، Campylobacter، و Clostridium difficile می‌توانند منجر به ایجاد اسهال چرب شوند.
4. مصرف دارو: مصرف برخی داروها مانند داروهای ضد التهاب غیر استروئیدی (NSAIDs) و داروهای ضد افسردگی می‌تواند منجر به ایجاد اسهال چرب شود.
5. مصرف غذاهای چرب: مصرف غذاهای چرب و چرب می‌تواند منجر به ایجاد اسهال چرب شود.
6. مشکلات گوارشی: مشکلات

In [44]:
text_gen('عوارض جانبی میزوپرستول چیست؟')

### Human:عوارض جانبی میزوپرستول چیست؟
### Assistant:عوارض جانبی میزوپرستول ممکن است شامل موارد زیر باشد:

1. تهوع و استفراغ: این عارضه جانبی در حدود 10-20 درصد از بیماران ایجاد می شود.
2. درد شکم: این عارضه جانبی در حدود 10-20 درصد از بیماران ایجاد می شود.
3. درد سرخرش: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
4. درد پشت: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
5. درد سر: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
6. درد پا: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
7. درد دست: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
8. درد کمر: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
9. درد شانه: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
10. درد بازو: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
11. درد ران: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
12. درد زانو: این عارضه جانبی در حدود 5-10 درصد از بیماران ایجاد می شود.
13. درد مچ پا: این عارضه ج

In [45]:
text_gen('برای درمان چه بیماری هایی از تنقیه استفاده می کنند؟')

### Human:برای درمان چه بیماری هایی از تنقیه استفاده می کنند؟
### Assistant:تنقیه به عنوان یک روش درمانی برای درمان چندین بیماری استفاده می شود. برخی از این بیماری ها عبارتند از:

1. آسم: تنقیه برای کنترل علائم آسم و کاهش نیاز به داروهای ضد آسم استفاده می شود.
2. آلرژی: تنقیه برای کاهش واکنش های آلرژیک و کاهش نیاز به داروهای ضد آلرژی استفاده می شود.
3. سرماخوردگی: تنقیه برای کاهش علائم سرماخوردگی و تسریع در بهبود استفاده می شود.
4. سرفه: تنقیه برای کاهش علائم سرفه و تسریع در بهبود استفاده می شود.
5. التهاب ریه: تنقیه برای کاهش علائم التهاب ریه و تسریع در بهبود استفاده می شود.
6. ذات الریه: تنقیه برای کاهش علائم ذات الریه و تسریع در بهبود استفاده می شود.
7. ذات الریه: تنقیه برای کاهش علائم ذات الریه و تسریع در بهبود استفاده می شود.
8. ذات الریه: تنقیه برای کاهش علائم ذات الریه و تسریع در بهبود استفاده می شود.

این تنها برخی از بیماری هایی است که تنقیه برای درمان آنها استفاده می شود. تنقیه می تواند برای درمان سایر بیماری ها نیز استفاده شود، اما باید با مشورت با یک پزشک انجام شود.


In [46]:
text_gen('فرق سندرم نفروتیک و نفریتیک در چیست؟')

### Human:فرق سندرم نفروتیک و نفریتیک در چیست؟
### Assistant:سندرم نفروتیک و نفریتیک دو شرایط پزشکی هستند که شبیه به هم هستند اما تفاوت های مهمی دارند.

سندرم نفروتیک یک شرایط پزشکی است که در آن کلیه ها به طور کامل یا جزئی از کار خود عوارض می کنند. این می تواند به دلیل عفونت، آسیب به کلیه ها، بیماری های کلیوی مزمن مانند دیابت یا بیماری کلیوی مزمن، یا سایر عوامل ایجاد شود. سندرم نفروتیک می تواند منجر به افزایش سدیم خون، کاهش پتاسیم خون، افزایش کلسیم خون، و سایر مشکلات سلامتی شود.

سندرم نفریتیک یک شرایط پزشکی است که در آن کلیه ها به طور کامل یا جزئی از کار خود عوارض می کنند، اما در این صورت کلیه ها به طور کامل از کار خود عوارض می کنند. این می تواند به دلیل عفونت، آسیب به کلیه ها، بیماری های کلیوی مزمن مانند دیابت یا بیماری کلیوی مزمن، یا سایر عوامل ایجاد شود. سندرم نفریتیک می تواند منجر به افزایش سدیم خون، کاهش پتاسیم خون، افزایش کلسیم خون، و سایر مشکلات سلامتی شود.

در حالی که سندرم نفروتیک و نفریتیک شبیه به هم هستند، سندرم نفریتیک شدیدتر است و می تواند منجر به مرگ شود. سندرم نفروتیک م

In [47]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline,
logging,
HfArgumentParser,
TrainingArguments
)
from peft import PeftModel, LoraConfig
from trl import SFTTrainer

In [48]:
data = pd.DataFrame({
    'text': b
})

In [49]:
data.head()

,text
0,### Human:دلیل ایجاد اسهال چرب چیست؟\n### Assi...
1,### Human:عوارض اسهال چرب چیست؟\n### Assistant...
2,### Human:درمان واریس مری چگونه انجام می شود؟\...
3,### Human:خطر اصلی و عوارض واریس مری چیست؟\n##...
4,### Human:عوارض جانبی میزوپرستول چیست؟\n### As...


In [50]:
train_dict = DatasetDict({'train': Dataset.from_pandas(data)})

In [51]:
train_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 50
    })
})

In [52]:
train_data = train_dict['train']

In [53]:
train_data

Dataset({
    features: ['text'],
    num_rows: 50
})

In [54]:
train_data[0]

{'text': '### Human:دلیل ایجاد اسهال چرب چیست؟\n### Assistant: علت آن عدم جذب صحیح چربی ها در روده باریک است. عدم جذب صحیح چربی ها منجر به افزایش میزان چربی در مدفوع می شود. افزایش میزان چربی مدفوع، باعث ایجاد حالتی شل در مدفوع می شود.'}

In [55]:
new_model = 'llama-3'

In [56]:
output_dir = './'

In [57]:
num_train_epochs = 40

In [58]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant = False
)

In [60]:
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_id,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [61]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [62]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation = True,
        padding = 'max_length',
        max_length = 512
    )

In [63]:
tokenized_dataset = train_data.map(preprocess_function, batched = True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [64]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [66]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 1,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [67]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments
)

Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [68]:
print('Starting training...')
trainer.train()
print('Training complete...')

Starting training...


Step,Training Loss
1,6.907300
2,7.130100
3,6.800200
4,6.676800
5,6.598500
6,6.839300
7,6.028400
8,5.007700
9,3.199000
10,1.064900


Training complete...


In [69]:
def text_gen(prompt):
  prompt = f"### Human:{prompt}\n### Assistant:"


  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

  generation_config = GenerationConfig(
      do_sample=True,
      top_k=1,
      temperature=0.99,
      max_new_tokens=600,
      pad_token_id=tokenizer.eos_token_id
  )


  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [70]:
text_gen('دلیل ایجاد اسهال چرب چیست؟')

### Human:دلیل ایجاد اسهال چرب چیست؟
### Assistant: علت آن عدم جذب صحیح چربی ها در روده باریک است. عدم جذب صحیح چربی ها منجر به افزایش میزان چربی در مدفوع می شود. افزایش میزان چربی مدفوع، باعث ایجاد حالتی شل در مدفوع می شود.


In [75]:
text_gen('عوارض جانبی میزوپرستول چیست؟')

### Human:عوارض جانبی میزوپرستول چیست؟
### Assistant: سردرد، تهوع، کرامپ رحمی، دل درد، اسهال، نفخ و تب؛ عوارض جدی نظیر پارگی رحم، اختلالات انعقادی و خونریزی شدید و غیرطبیعی واژینال.


In [77]:
text_gen('برای درمان چه بیماری هایی از تنقیه استفاده می کنند؟')

### Human:برای درمان چه بیماری هایی از تنقیه استفاده می کنند؟
### Assistant: گاه برای درمان بیماریهای گوارشی مانند یبوست شدید یا ولولوس استفاده می شود.


In [79]:
text_gen('فرق سندرم نفروتیک و نفریتیک در چیست؟')

### Human:فرق سندرم نفروتیک و نفریتیک در چیست؟
### Assistant: در سندرم نفریتیک علاوه بر دفع پروتئین٬ دفع خون از طریق ادرار وجود دارد.


In [81]:
text_gen('سلام خوب هستی؟ می توانی فرق سندرم نفروتیک و نفریتیک را توضیح دهی؟')

### Human:سلام خوب هستی؟ می توانی فرق سندرم نفروتیک و نفریتیک را توضیح دهی؟
### Assistant: بله توضیح می کنم. نفریتیک یک نوع ضعف در تشکیل خون است و به طور دائمی وجود دارد در حالی که سندرم نفروتیک یک واکنش ایمنی است که بعد از وجود عفونت یا مواد خارجی ایجاد می شود و می تواند در هر زمان وجود داشته باشد. در نفریتیک خونریزی کلیوی وجود دارد اما در واکنش ایمنی نیز می توانیم ببینیم که روی خون و هموگلوبین هدف هستند.


In [82]:
text_gen('می خواستم بدانم برای درمان چه بیماری هایی از تنقیه استفاده می شود؟')

### Human:می خواستم بدانم برای درمان چه بیماری هایی از تنقیه استفاده می شود؟
### Assistant: گاه برای درمان بیماریهای گوارشی و دلدیدی استفاده می شود.


In [83]:
text_gen('چرا از تنقیه استفاده می کنند؟')

### Human:چرا از تنقیه استفاده می کنند؟
### Assistant: برای پاک کردن روده از مواد شیمیایی در روده پاک کننده استفاده می کنند و این روش برای کسانی استفاده می شود که از خطر حمله از خارج دردهای خود تهدید نمی شوند.


In [85]:
text_gen('میزوپرستول چه عوارض جانبی به همراه دارد؟')

### Human:میزوپرستول چه عوارض جانبی به همراه دارد؟
### Assistant: به طور معمول این دارو به همراه برخی از عوارض جانبی مانند اسهال شدید، اسهال خونین، نفخ، درد عضلانی و کلفت می باشد.


In [88]:
text_gen('دلایل ایجاد اسهال چرب چیست؟')

### Human:دلایل ایجاد اسهال چرب چیست؟
### Assistant: اسهال چرب ممکن است ناشی از اختلال در جذب چربی باشد که در حین جذب مواد غذایی ایجاد می شود. همچنین اختلال در تولید و ترشح چربی در روده ALSO
